In [40]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
from IPython.display import display, clear_output
from datetime import datetime
import pandas as pd

In [41]:
# Load datasets
stock_df = pd.read_csv('data/big_tech_stock_prices.csv')
timeline_df = pd.read_csv('data/timeline.csv', delimiter=',', on_bad_lines='skip')

# Convert date columns to datetime
stock_df['date'] = pd.to_datetime(stock_df['date'])

# Filter timeline data to match stock data timeframe (2010-2023)
timeline_df['date'] = pd.to_datetime(timeline_df['date'], format='%Y', errors='coerce')
timeline_df = timeline_df[
    (timeline_df['date'] >= '2010-01-01') & 
    (timeline_df['date'] <= '2023-12-31')
].copy()

# Handle missing values in stock data
stock_df = stock_df.dropna(subset=['open', 'high', 'low', 'close'])

# Get list of unique stock symbols
stock_symbols = stock_df['stock_symbol'].unique()

In [42]:
def generate_candlestick_chart(symbol, days):
    # Filter stock data for the selected company
    company_data = stock_df[stock_df['stock_symbol'] == symbol]
    
    # Determine date range: for 'Max', use the full range; otherwise use last N days
    end_date = company_data['date'].max()
    start_date = company_data['date'].min() if days == 5000 else end_date - pd.Timedelta(days=days)
    filtered_data = company_data[(company_data['date'] >= start_date) & (company_data['date'] <= end_date)]
    
    # Create figure with two subplots (Price and Volume)
    fig = make_subplots(
        rows=2, cols=1, 
        shared_xaxes=True,
        vertical_spacing=0.03,
        row_heights=[0.7, 0.3]
    )
    
    # Add candlestick trace for price
    fig.add_trace(
        go.Candlestick(
            x=filtered_data['date'],
            open=filtered_data['open'],
            high=filtered_data['high'],
            low=filtered_data['low'],
            close=filtered_data['close'],
            name=symbol
        ),
        row=1, col=1
    )
    
    # Add volume bar chart in the second row
    fig.add_trace(
        go.Bar(
            x=filtered_data['date'],
            y=filtered_data['volume'],
            name='Volume',
            marker_color='rgba(128, 128, 128, 0.5)'
        ),
        row=2, col=1
    )
    
    # Filter timeline events for the selected company and time period
    company_events = timeline_df[
        (timeline_df['stock_symbol'] == symbol) & 
        (timeline_df['date'] >= start_date) & 
        (timeline_df['date'] <= end_date)
    ]
    
    # Add event markers (vertical dashed red lines) and annotations
    shapes = []
    annotations = []
    for _, event in company_events.iterrows():
        shapes.append(dict(
            type='line',
            x0=event['date'],
            x1=event['date'],
            y0=0,
            y1=1,
            xref='x paper',
            yref='paper',
            line=dict(color='red', width=1, dash='dot')
        ))
        annotations.append(dict(
            x=event['date'],
            y=1.0,
            xref='x',
            yref='paper',
            showarrow=True,
            arrowhead=2,
            arrowcolor='red',
            arrowsize=1,
            arrowwidth=1,
            text=event['description'],
            bgcolor='rgba(255, 255, 255, 0.8)',
            bordercolor='red',
            borderwidth=1,
            borderpad=4,
            font=dict(size=10)
        ))
    

    fig.update_layout(
        title=f'{symbol} Stock Price',
        xaxis_title='Date',
        yaxis_title='Price',
        yaxis2_title='Volume',
        shapes=shapes,
        annotations=annotations,
        xaxis_rangeslider_visible=False,
        height=700,
        showlegend=True,
        legend=dict(orientation='h', yanchor='bottom', y=1.02, xanchor='right', x=1),
        margin=dict(l=50, r=50, t=85, b=50),
        hovermode='x unified',
        dragmode='zoom',
        selectdirection='h'
    )
    
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="Volume", row=2, col=1)
    
    return fig

In [44]:
from dash import dcc, html, Input, Output
from jupyter_dash import JupyterDash

# Initialize Dash app
app = JupyterDash(__name__)
app.layout = html.Div([
    html.H1("Big Tech Stock Dashboard"),
    html.Div([
        html.Div([
            html.Label("Company:"),
            dcc.Dropdown(
                id='company-dropdown',
                options=[{'label': symbol, 'value': symbol} for symbol in stock_symbols],
                value=stock_symbols[0],
                clearable=False,
                style={'width': '90%'}
            )
        ], style={'flex': '1', 'padding': '10px'}),
        html.Div([
            html.Label("Time Period:"),
            dcc.Dropdown(
                id='period-dropdown',
                options=[
                    {'label': '7 Days', 'value': 7},
                    {'label': '30 Days', 'value': 30},
                    {'label': '90 Days', 'value': 90},
                    {'label': '1 Year', 'value': 365},
                    {'label': 'Max', 'value': 5000}
                ],
                value=30,
                clearable=False,
                style={'width': '90%'}
            )
        ], style={'flex': '1', 'padding': '10px'})
    ], style={'display': 'flex', 'flex-direction': 'row', 'justify-content': 'space-around'}),
    
    dcc.Graph(id='candlestick-chart')
])

@app.callback(
    Output('candlestick-chart', 'figure'),
    [Input('company-dropdown', 'value'),
     Input('period-dropdown', 'value')]
)
def update_chart(symbol, period):
    return generate_candlestick_chart(symbol, period)

app.run(mode='inline', debug=True)